In [31]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
import xgboost as xgb

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder, label_binarize
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten

In [33]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [34]:
def create_convnext_like_feature_extractor(input_dim, sliding_window_size=3):
    input_layer = Input(shape=(input_dim,))
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)
    x = reshaped_input

    # Simulated ConvNeXt block
    x = Conv1D(filters=int(max(16, input_dim * 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 4)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 8)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)

    # Flatten the output for feature extraction
    feature_output = Flatten()(x)

    # Reconstruction head for self-supervised learning
    reconstruction_output = Dense(input_dim, activation='linear')(feature_output)

    # Create the model
    model = Model(inputs=input_layer, outputs=reconstruction_output)
    return model

In [35]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(['label'], axis=1)
    train_y = train_data['label'] - 1
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(['label'], axis=1)
    test_y = test_data['label'] - 1
    del test_data
    gc.collect()

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Define XGBoost parameters
    params = {
        "objective": "multi:softprob",
        "num_class": len(train_y.unique()),
        "eval_metric": "mlogloss",
        "max_depth": 6,
        "eta": 0.3,
        "seed": 42
    }

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs} started...")
                
        # Train the NN feature extractor
        feature_extractor = create_convnext_like_feature_extractor(train_X.shape[1])
        feature_extractor.compile(optimizer='adam', loss='mean_squared_error')
        feature_extractor.fit(train_X, train_X, epochs=3, batch_size=32, verbose=0)  # Self-supervised training

        # Extract features
        feature_extractor_truncated = Model(inputs=feature_extractor.input, outputs=feature_extractor.layers[-2].output)
        train_features = feature_extractor_truncated.predict(train_X, verbose=0)
        test_features = feature_extractor_truncated.predict(test_X, verbose=0)

        # Convert data to DMatrix
        dtrain = xgb.DMatrix(data=train_features, label=train_y)
        dtest = xgb.DMatrix(data=test_features, label=test_y)

        # Train XGBoost
        num_round = 100
        bst = xgb.train(params, dtrain, num_round)

        # Make predictions
        test_probabilities = bst.predict(dtest)
        test_predictions = np.argmax(test_probabilities, axis=1)
        del feature_extractor
        gc.collect()

        test_y_binarized = label_binarize(test_y, classes=range(len(np.unique(test_y))))

        # Compute metrics
        accuracy = accuracy_score(test_y, test_predictions)
        balanced_acc = balanced_accuracy_score(test_y, test_predictions)
        roc_auc = roc_auc_score(test_y, test_probabilities, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc_scores = []
        for i in range(test_y_binarized.shape[1]):  # Iterate over each class
            pr_auc = average_precision_score(test_y_binarized[:, i], test_probabilities[:, i])
            pr_auc_scores.append(pr_auc)
        pr_auc = np.mean(pr_auc_scores)
        f2 = fbeta_score(test_y, test_predictions, beta=2, average='weighted')
        mcc = matthews_corrcoef(test_y, test_predictions)

        # Store metrics
        metrics_storage['test_accuracy'].append(accuracy)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(test_y, test_predictions, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [ ]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ma-bsmote/shuttle_train.csv'

In [ ]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

In [ ]:
run_model("/kaggle/input/ma-datasets/kdd_train.csv", "/kaggle/input/ma-datasets/kdd_test.csv", is_string_labels = False)

In [ ]:
run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = False)